# Notebook 0: Setup & Configuration

Welcome to the Prompt Engineering for Mistral on AWS Bedrock tutorial series!

## What You'll Learn in This Series

This tutorial will teach you how to craft effective prompts for Mistral models. You'll learn:

- How to structure prompts with system and user messages
- Techniques for defining roles and purposes
- How to organize complex instructions clearly
- Using delimiters and formatting for safety and clarity
- Few-shot prompting to guide model behavior
- Controlling output formats for reliable parsing
- Step-by-step reasoning for complex tasks
- Reducing hallucinations and grounding responses

## Prerequisites

- AWS account with Bedrock access
- Mistral models enabled in your Bedrock console ([enable here](https://console.aws.amazon.com/bedrock/home#/modelaccess))
- Python 3.8+
- Basic Python knowledge

## Reference

- [Mistral Prompting Documentation](https://docs.mistral.ai/guides/prompting/)

---
## Section 1: Install Dependencies

In [ ]:
%pip install boto3 --quiet

---
## Section 2: Configuration

### Model Selection

We use **Ministral 14B** (`mistral.ministral-3-14b-instruct`) for this tutorial series.

In [ ]:
# =============================================================================
# CONFIGURATION - Modify these settings as needed
# =============================================================================

# Model ID - change this to switch models
MODEL_ID = "mistral.ministral-3-14b-instruct"

# AWS Region - must support Mistral models
# Supported regions for Ministral: us-east-1, us-east-2, us-west-2, eu-west-1, eu-west-2, ap-northeast-1, ap-south-1, sa-east-1
AWS_REGION = "us-west-2"

# Default inference parameters
DEFAULT_MAX_TOKENS = 1024
DEFAULT_TEMPERATURE = 0.5

### AWS Credentials

This notebook assumes you have AWS credentials configured via one of these methods:

1. **SageMaker Notebook**: Credentials are automatic via the execution role
2. **Environment variables**: `AWS_ACCESS_KEY_ID` and `AWS_SECRET_ACCESS_KEY`
3. **AWS CLI**: Run `aws configure` in your terminal
4. **IAM Role**: If running on EC2 with an attached role

Your credentials need the `bedrock:InvokeModel` permission.

---
## Section 3: Initialize the Bedrock Client

In [ ]:
import boto3
import json

# Create the Bedrock Runtime client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"Bedrock client initialized for region: {AWS_REGION}")

---
## Section 4: Helper Function

This helper function wraps the Bedrock API call. You'll use it throughout all notebooks.

In [ ]:
def call_mistral(
    user_prompt: str,
    system_prompt: str = None,
    model_id: str = MODEL_ID,
    max_tokens: int = DEFAULT_MAX_TOKENS,
    temperature: float = DEFAULT_TEMPERATURE
) -> str:
    """
    Call a Mistral model on Bedrock using the Converse API.

    Args:
        user_prompt: The user message to send
        system_prompt: Optional system prompt to set model behavior
        model_id: The Bedrock model ID to use
        max_tokens: Maximum tokens in the response
        temperature: Sampling temperature (0-1)

    Returns:
        The model's response text
    """
    # Build messages list for Converse API
    messages = [
        {
            "role": "user",
            "content": [{"text": user_prompt}]
        }
    ]

    # Build inference config
    inference_config = {
        "maxTokens": max_tokens,
        "temperature": temperature
    }

    # Build request parameters
    request_params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": inference_config
    }

    # Add system prompt if provided
    if system_prompt:
        request_params["system"] = [{"text": system_prompt}]

    # Call the model using Converse API
    response = bedrock_client.converse(**request_params)

    # Extract and return the response text with robust error handling
    try:
        content = response["output"]["message"]["content"][0]
        if "text" in content:
            return content["text"]
        else:
            # Handle other content types (e.g., toolUse)
            return str(content)
    except (KeyError, IndexError) as e:
        # Return a descriptive error message if response structure is unexpected
        return f"[Error extracting response: {e}. Raw response: {response}]"


def call_mistral_with_messages(
    messages: list,
    system_prompt: str = None,
    model_id: str = MODEL_ID,
    max_tokens: int = DEFAULT_MAX_TOKENS,
    temperature: float = DEFAULT_TEMPERATURE
) -> str:
    """
    Call a Mistral model with a full messages list (for few-shot prompting).
    Uses the Converse API.

    Args:
        messages: List of message dicts with 'role' and 'content' (content as string)
                  Supports 'system', 'user', and 'assistant' roles.
                  System messages are automatically extracted and passed separately.
        system_prompt: Optional additional system prompt
        model_id: The Bedrock model ID to use
        max_tokens: Maximum tokens in the response
        temperature: Sampling temperature (0-1)

    Returns:
        The model's response text
    """
    # Extract system messages and non-system messages
    system_messages = []
    converse_messages = []
    
    for msg in messages:
        if msg["role"] == "system":
            # Collect system message content
            system_messages.append(msg["content"])
        else:
            # Convert to Converse API format
            converse_messages.append({
                "role": msg["role"],
                "content": [{"text": msg["content"]}]
            })
    
    # Combine system messages with any provided system_prompt
    combined_system = []
    if system_prompt:
        combined_system.append(system_prompt)
    combined_system.extend(system_messages)
    
    # Build inference config
    inference_config = {
        "maxTokens": max_tokens,
        "temperature": temperature
    }

    # Build request parameters
    request_params = {
        "modelId": model_id,
        "messages": converse_messages,
        "inferenceConfig": inference_config
    }

    # Add combined system prompt if any
    if combined_system:
        request_params["system"] = [{"text": "\n\n".join(combined_system)}]

    # Call the model using Converse API
    response = bedrock_client.converse(**request_params)

    # Extract and return the response text with robust error handling
    try:
        content = response["output"]["message"]["content"][0]
        if "text" in content:
            return content["text"]
        else:
            # Handle other content types (e.g., toolUse)
            return str(content)
    except (KeyError, IndexError) as e:
        # Return a descriptive error message if response structure is unexpected
        return f"[Error extracting response: {e}. Raw response: {response}]"


print("Helper functions defined successfully!")

---
## Section 5: Test Your Setup

Let's verify everything works with a simple test.

In [ ]:
# Simple test - the model should respond with a single word
response = call_mistral(
    user_prompt="Respond with exactly one word: Hello",
    temperature=0.0  # Use 0 for deterministic output
)

print(f"Model: {MODEL_ID}")
print(f"Response: {response}")
print("\n✅ Setup complete! You're ready to start the tutorial.")

---
## Section 6: Understanding the Response Structure

Let's look at what comes back from Bedrock in more detail.

In [ ]:
def call_mistral_raw(
    user_prompt: str,
    system_prompt: str = None,
    model_id: str = MODEL_ID,
    max_tokens: int = DEFAULT_MAX_TOKENS,
    temperature: float = DEFAULT_TEMPERATURE
) -> dict:
    """
    Call a Mistral model using Converse API and return the full response.
    """
    # Build messages list for Converse API
    messages = [
        {
            "role": "user",
            "content": [{"text": user_prompt}]
        }
    ]

    # Build inference config
    inference_config = {
        "maxTokens": max_tokens,
        "temperature": temperature
    }

    # Build request parameters
    request_params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": inference_config
    }

    # Add system prompt if provided
    if system_prompt:
        request_params["system"] = [{"text": system_prompt}]

    # Call the model using Converse API
    response = bedrock_client.converse(**request_params)

    return response


# Get the full response
full_response = call_mistral_raw(
    user_prompt="What is 2 + 2?",
    temperature=0.0
)

print("Full Converse API response structure:")
print(json.dumps(full_response, indent=2, default=str))

The Converse API response includes:
- `output.message.content`: List of content blocks (usually one with `text`)
- `output.message.role`: Will be "assistant"
- `usage`: Token counts (`inputTokens` and `outputTokens`)
- `stopReason`: Why generation stopped (e.g., "end_turn", "max_tokens")

---
## Exercise: Adjust Temperature

Temperature controls randomness. Lower = more deterministic, higher = more creative.

Run the same prompt multiple times with different temperatures to see the effect.

In [ ]:
prompt = "Write a one-sentence description of a cat."

print("=" * 50)
print("Temperature: 0.0 (deterministic)")
print("=" * 50)
for i in range(3):
    response = call_mistral(prompt, temperature=0.0)
    print(f"Run {i+1}: {response}")

print("\n" + "=" * 50)
print("Temperature: 1.0 (creative)")
print("=" * 50)
for i in range(3):
    response = call_mistral(prompt, temperature=1.0)
    print(f"Run {i+1}: {response}")

---
## Next Steps

Your setup is complete! Proceed to **Notebook 1: Basic Prompt Structure** to start learning prompt engineering techniques.

📚 [Continue to Notebook 1 →](01_basic_prompt_structure.ipynb)